# Tavily Research with MCP

In this notebook, we'll walk you through the process of performing research with Tavily with an MCP server.

By passing in an MCP server you can seemlessly use our API to blend your internal data and external data sources with the Tavily web access. We will use the tools within the MCP servers along with our web access to get in depth research ready for you in production.


In [ ]:
%pip install -q tavily-python

In [ ]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY:\n")

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [ ]:
from tavily import TavilyClient
import time

tavily_client = TavilyClient()

## MongoDB Mcp

By including an MCP server like mongo which can search within your internal data, the researcher can now look to the web and to your internal data to maximize research effectiveness.


In [ ]:
response = tavily_client.research(
    task="I'm looking for a good movie to watch. I'm in the mood for something that has action, but also is funny. Find me a list of 10 movies with summaries and genres.",
    mcps=[{
        "name": "", # the name you want to give this mcp connection
        "url": "", # the mongo mcp url to connect to. This will be your local or deployed mongo MCP instance (https://github.com/mongodb-js/mongodb-mcp-server)
        "transport": "streamable_http",
        "headers": {
            "MDB_MCP_API_CLIENT_ID": "",
            "MDB_MCP_API_CLIENT_SECRET": "",
            "MDB_MCP_CONNECTION_STRING": ""
        }
    }]
)


research_id = response.id

while True:
    result = tavily_client.get_research(research_id)

    if result.status == "completed":
        research_report = result.content
        sources = result.sources
        break

    if result.status == "failed":
        raise RuntimeError(f"Research failed: {result.error}")

    print(f"Status: {result.status}... polling again in 5 seconds")
    time.sleep(5)

In [ ]:
research_report
sources

## Now here's an example using the Github remote MCP

You will need a PAT from your github to connect to your personal github.


In [ ]:
full_name = "YOUR_FULL_NAME"
response = tavily_client.research(
    task_description=f"Tell me about {full_name}. I want to know his experience, education. Also his github usage, I want to know his projects, commit histories and how he iterates on projects.",
    mcps=[
        {
            "name": "Github",
            "url": "https://api.githubcopilot.com/mcp/",
            "transport": "streamable_http",
            "headers": {
            "Authorization": "Bearer github_pat_xxxxx"
            }
        }
    ]
)

research_id = response.id

while True:
    result = tavily_client.get_research(research_id)

    if result.status == "completed":
        research_report = result.content
        sources = result.sources
        break

    if result.status == "failed":
        raise RuntimeError(f"Research failed: {result.error}")

    print(f"Status: {result.status}... polling again in 5 seconds")
    time.sleep(5)

In [ ]:
research_report
sources